In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
training_data = datasets.MNIST(
    root="data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data", train=False, download=True, transform=ToTensor()
)

train_dataloader = torch.utils.data.DataLoader(
    training_data, batch_size=10, shuffle=True
)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=10, shuffle=True)

In [ ]:
img, label = training_data[1]
plt.axis("off")
plt.imshow(img.squeeze(), cmap="gray")
print(img)

In [ ]:
img.shape

In [ ]:
training_data.classes

In [ ]:
train_labels = training_data.train_labels

In [ ]:
labels_counts = np.unique(train_labels.numpy(), return_counts=True)
x_values = []
for idx, v in enumerate(labels_counts[0]):
    x_values.append(str(v))

print(x_values)
plt.bar(x_values, labels_counts[1], color="red")
plt.xlabel("Label")
plt.ylabel("Number of samples")
plt.title("MNIST Dataset Analysis")

In [ ]:
print("GPU: {}", torch.cuda.is_available())
print("MPS: {}", torch.backends.mps.is_available())

In [ ]:
device = torch.device("cpu")
device

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
import lightning as L
from lightning.pytorch.utilities.types import STEP_OUTPUT, OptimizerLRScheduler
from torch import nn
import torch.nn.functional as F
import torchmetrics


class LightningNeuralNetwork(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # nn.Dropout(0.2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(16 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
        )

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.001)
        return optimizer

    def training_step(self, batch, batch_idx):
        inputs, labels = batch

        # Make predictions for this batch
        outputs = self.model(inputs)

        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        self.log("train_loss", loss, on_epoch=True)
        accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(device)
        self.log("train_acc", accuracy(outputs, labels))
        return loss

    def validation_step(self, val_batch, batch_idx):
        inputs, labels = val_batch
        outputs = self.model(inputs)
        loss = criterion(outputs, labels)
        self.log("val_loss", loss)
        accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(device)
        self.log("val_acc", accuracy(outputs, labels))

In [ ]:
model = LightningNeuralNetwork()
trainer = L.Trainer(accelerator=device.type, devices=1, max_epochs=11)
trainer.fit(model, train_dataloader, test_dataloader)

In [ ]:
def make_prediction(x):
    with torch.no_grad():
        prediction = model(x)
        classes = training_data.classes
        return classes[np.argmax(prediction)]

In [ ]:
from PIL import Image

my_image = Image.open("./My images/Zrzut ekranu 2023-11-12 o 03.00.06.png")
my_image = my_image.resize((28, 28))
my_image_array = np.array(my_image)
my_image_to_recognize = my_image_array[:, :, 0].reshape(1, 28, 28)
my_image_to_recognize = (my_image_to_recognize - np.min(my_image_to_recognize)) / (
    np.max(my_image_to_recognize) - np.min(my_image_to_recognize)
)
my_image_to_recognize = 1 - my_image_to_recognize
print(my_image_array.shape)
print(my_image_to_recognize.shape)
plt.imshow(my_image_to_recognize.squeeze(), cmap="gray")

In [ ]:
model.eval()
image_tensor = torch.from_numpy(my_image_to_recognize).float().unsqueeze(0).to(device)
print(image_tensor.shape)
print(image_tensor)

In [ ]:
print(make_prediction(image_tensor))